# Jupyter Notebook pour des prédictions législatives

## 1. Constantes

In [323]:
UNION_GAUCHE = True
UNION_DROITE = False
REPORT_MACRON_GAUCHE = 0.5
REPORT_MACRON_DROITE = 0.5
REPORT_LR_MACRON = 0.4
REPORT_LR_RN = 0.6

In [324]:
TABLE_LEGISLATIVES = {
    "DXG" : "Marxistes/Divers Extrèmes gauche",
    "REG" : "Extrème gauche",
    "NUP" : "Nupes",
    "DVG" : "Communistes/Divers gauches",
    "ECO" : "Ecologistes",
    "PS" : "Parti Socialiste",
    "RDG" : "Radicaux de gauche (Centre gauche)",
    "DVC" : "Divers Centre",
    "ENS" : "Renaissance",
    "UDI" : "Union des démocrates et indépendants (Centre droit)",
    "LR" : "Les républicains",
    "DVD" : "Divers droites",
    "REC" : "Reconquête",
    "DSV" : "Dupont Aignant",
    "DXD" : "Divers Extrème droite",
    "DIV" : "Divers",
    "LLR" : "Lassale",
}

TABLE_EUROPEENNES = {
    "PARTI REVOLUTIONNAIRE COMMUNISTES" : "DXG",
    "FREE PALESTINE" : "DXG",
    "HUMANITE SOUVERAINE" : "DXG",
    "PAIX ET DECROISSANCE" : "DXG",
    "POUR UNE DEMOCRATIE REELLE : DECIDONS NOUS-MEMES !" : "DXG",
    "NON À L'UE ET À L'OTAN, COMMUNISTES POUR LA PAIX ET LE PROGRÈS SOCIAL" : "DXG",
    "LUTTE OUVRIERE" : "DXG",
    "URGENCE REVOLUTION !" : "DXG",
    "LFI - UP" : "NUP",
    '"POUR LE PAIN, LA PAIX, LA LIBERTÉ !" PRÉSENTÉE PAR LE PARTI DES TRAVAILLEURS' : "DVG",
    "PARTI ANIMALISTE" : "DVG",
    "PACE" : "DVG",
    "GAUCHE UNIE" : "DVG",
    "REVEIL EUR" : "PS",
    "EUROPE ÉCOLOGIE" : "EELV",
    "ÉQUINOXE" : "ECO",
    "ECOLOGIE POSITIVE" : "ECO",
    "EUROPE TERRITOIRES ÉCOLOGIE" : "ECO",
    "ÉCOLOGIE AU CENTRE" : "ECO",
    "BESOIN D'EUROPE" : "ENS",
    "LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FRANCE EN EUROPE" : "LR",
    "LA RUCHE CITOYENNE" : "DVC",
    "DEMOCRATIE REPRESENTATIVE" : "DVC",
    "PARTI PIRATE" : "DVC",
    "LRN" : "RN",
    "FORTERESSE EUROPE": "DVD",
    "LIBERTÉ DÉMOCRATIQUE FRANÇAISE" : "DVD",
    "PPL" : "DVD",
    "La FRANCE REVIENT" : "RN",
    "LA FRANCE FIERE, MENEE PAR MARION MARECHAL ET SOUTENUE PAR ÉRIC ZEMMOUR" : "REC",
    "L'EUROPE CA SUFFIT !" : "DSV",
    "LISTE ASSELINEAU-FREXIT" : "DSV",
    "POUR UNE AUTRE EUROPE" : "DIV",
    "NOUS LE PEUPLE" : "DIV",
    "PRENONS-NOUS EN MAIN" : "DIV",
    "CHANGER L'EUROPE" : "DIV",
    "FRANCE LIBRE" : "DIV",
    "DEFENDRE LES ENFANTS" : "DIV",
    "ESPERANTO" : "DIV",
    "NLP" : "DIV",
    "EAC" : "DIV",
    "AR" : "LLR",
}

## 2. Data

### 2.1 Importer la data

In [325]:
import pandas as pd

# import data
df = pd.read_excel('../data/leg_tour_2.xlsx')
df_leg_1 = pd.read_excel('../data/leg_tour_1.xlsx')
df_euro = pd.read_excel('../data/europeennes_circ.xlsx')


### 2.2 Traitement de la data législative

In [326]:
# Identify the candidate that lost
df['win_party'] = df.apply(lambda row: row["Nuance_2"] if row['Siege_1'] != 'Elu' else row["Nuance_1"], axis=1)

print(df["win_party"].value_counts())

win_party
ENS    245
NUP    127
RN      88
LR      60
DVG     23
DVD     10
REG     10
DVC      4
UDI      3
DSV      1
DIV      1
Name: count, dtype: int64


### 2.3 Traitement de la data européenne

In [327]:
df_euro['win_party'] = ""
for index, row in df_euro.iterrows():
    win_party = row["Libellé abrégé de liste 1"]
    max_voix = -1
    for i in range (1, 39):
        if row[f"Voix {i}"] > max_voix:
            max_voix = row[f"Voix {i}"]
            win_party = row[f"Libellé abrégé de liste {i}"]
    df_euro.loc[index, "win_party"] = win_party
print(df_euro["win_party"].value_counts())

win_party
La FRANCE REVIENT                                                          457
LFI - UP                                                                    48
BESOIN D'EUROPE                                                             39
REVEIL EUR                                                                  30
LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FRANCE EN EUROPE                 2
LA FRANCE FIERE, MENEE PAR MARION MARECHAL ET SOUTENUE PAR ÉRIC ZEMMOUR      1
Name: count, dtype: int64


In [328]:
ALLIANCES_PARTY = {
    "DXG" : "Extrème gauche",
    "REG" : "Extrème gauche",
    "DVG" : "Extrème gauche",
    "NUP" : "Alliance gauche",
    "ECO" : "Alliance gauche",
    "PS" : "Alliance gauche",
    "RDG" : "Alliance gauche",
    "DVC" : "Alliance gauche",
    "UDI" : "Alliance gauche",
    "ENS" : "Renaissance",
    "LR" : "Alliance droite",
    "DVD" : "Alliance droite",
    "REC" : "Extrème droite",
    "DSV" : "Extrème droite",
    "DXD" : "Extrème droite",
    "DIV" : "Divers",
    "LLR" : "Divers",
}

## 3. Elections

### 3.1 Préparer les candidats

### Hypothèse

#### à Gauche

Le PS présente partout là il a gagné les européennes
Les verts présentent là où ils ont gagné les européennes
Le reste c LFI

#### La droite

LR tous les sortants sont remis
RN le reste
Reconquête le reste aussi

In [331]:
import pandas as pd

# Création de la table des candidats.
ALLIANCES_PARTY = {
    "DXG" : "Extrème gauche",
    "REG" : "Extrème gauche",
    "DVG" : "Extrème gauche",
    "NUP" : "Alliance gauche",
    "LFI - UP" : "Alliance gauche",
    "ECO" : "Alliance gauche",
    "PS" : "Alliance gauche",
    "EELV" : "Alliance gauche",
    "RDG" : "Alliance gauche",
    "DVC" : "Alliance gauche",
    "UDI" : "Alliance gauche",
    "ENS" : "Macron",
    "LR" : "Alliance droite",
    "DVD" : "Alliance droite",
    "RN" : "Alliance droite",
    "REC" : "Extrème droite",
    "DSV" : "Extrème droite",
    "DXD" : "Extrème droite",
    "DIV" : "Divers",
    "LLR" : "Divers",
}
# Create a new DataFrame with the specified columns
df_candidats = pd.DataFrame(columns=[
    "Libellé du département",
    "Libellé de la circonscription",
    "Extrème gauche", 
    "Alliance gauche",
    "Macron",
    "Alliance droite",
    "Extrème droite",
    "Divers",
    "Votes Extrème gauche", 
    "Votes Alliance gauche",
    "Votes Macron",
    "Votes Alliance droite",
    "Votes Extrème droite", 
    "Votes Divers",   
])

# pour une raison que j'ignore, ma table des législatives n'a pas toutes les rows mdr
for index, row in df.iterrows():
# Determine the winning party
    winning_party = row["win_party"]

    # Get the alliance of the winning party
    alliance = ALLIANCES_PARTY[winning_party]

    # Create the new row
    new_row = {
        "Libellé du département" : row["Libellé du département"],
        "Libellé de la circonscription" : row["Libellé de la circonscription"],
        "Extrème gauche": "", 
        "Alliance gauche": "",
        "Macron": "",
        "Alliance droite": "",
        "Extrème droite": "",
        "Divers": "",
        "Votes Extrème gauche": 0, 
        "Votes Alliance gauche": 0,
        "Votes Macron": 0,
        "Votes Alliance droite": 0,
        "Votes Extrème droite": 0, 
        "Votes Divers": 0,
    }
    new_row[alliance] = winning_party

    # On regarde qui mettre si l'alliance de gauche n'a pas de candidats : 
    winning_party = df_euro.loc[index, "win_party"]
    if new_row["Alliance gauche"] == "":
        # Regarder qui a gagné aux européennes
        if ALLIANCES_PARTY[TABLE_EUROPEENNES[winning_party]] == "Alliance gauche":
            new_row["Alliance gauche"] = winning_party
        else:
            new_row["Alliance gauche"] = "NUP"

    # On regarde qui mettre si l'alliance de droite n'a pas de candidats :
    if new_row["Alliance droite"] == "":
        # Regarder qui a gagné aux européennes
        if ALLIANCES_PARTY[TABLE_EUROPEENNES[winning_party]] == "Alliance droite":
            new_row["Alliance droite"] = winning_party
        else:
            new_row["Alliance droite"] = "RN"
    
    if new_row["Macron"] == "":
        new_row["Macron"] = "ENS"

    if new_row["Extrème gauche"] == "":
        # On regarde qui a gagné aux européennes
        if ALLIANCES_PARTY[TABLE_EUROPEENNES[winning_party]] == "Extrème gauche":
            new_row["Extrème gauche"] = winning_party
        else:
            new_row["Extrème gauche"] = "DXG"

    if new_row["Extrème droite"] == "":
        # On regarde qui a gagné aux européennes
        if ALLIANCES_PARTY[TABLE_EUROPEENNES[winning_party]] == "Extrème droite":
            new_row["Extrème droite"] = winning_party
        else:
            new_row["Extrème droite"] = "DXD"
    
    if new_row["Divers"] == "":
        # On regarde qui a gagné aux européennes
        if ALLIANCES_PARTY[TABLE_EUROPEENNES[winning_party]] == "Divers":
            new_row["Divers"] = winning_party
        else:
            new_row["Divers"] = "DIV"

    # Add the new row to df_candidats
    df_candidats = pd.concat([df_candidats, pd.DataFrame([new_row])], ignore_index=True)

### 3.2 Premier tour

Pour ça, on se focalise uniquement sur ça :
On regarde le nombre d'électeurs qui ont voté aux européennes, ceux là ne changeront pas leur vote
On regarde ensuite le nombre d'électeurs qui ont voté aux législatives, on fait une différence, et on prend leur vote, qu'on ajoute (on considère qu'ils vont se bouger le cul pour les législatives)

In [332]:
for index, row in df_candidats.iterrows():
    # Add the votes of the européennes
    # Obtenir le nombre de votants des législatives
    row_leg = df_leg_1.loc[(df_leg_1["Libellé du département"] == row["Libellé du département"]) & (df_leg_1["Libellé de la circonscription"] == row["Libellé de la circonscription"])]
    row_eur = df_euro.loc[(df_euro["Libellé département"] == row["Libellé du département"]) & (df_euro["Libellé circonscription législative"] == row["Libellé de la circonscription"])]

    nb_votants_leg = row_leg["Exprimés"].iloc[0]
    if row_eur.empty:
        continue
    nb_votants_eur = row_eur["Exprimés"].iloc[0]
    
    # Ajouter les votes obtenus aux européennes   
    for i in range(1, 39):
        party = row_eur[f"Libellé abrégé de liste {i}"].iloc[0]
        party = TABLE_EUROPEENNES[party]
        alliance = ALLIANCES_PARTY[party]
        df_candidats.at[index, f"Votes {alliance}"] += row_eur[f"Voix {i}"].iloc[0]    # Ajouter les votes des législatives pour les gens n'étant pas venus aux européennes

    # Ajouter les votes législatifs
    if (nb_votants_leg >= nb_votants_eur):
        pourcentage = (nb_votants_leg - nb_votants_eur) / nb_votants_leg
        for i in range (1, 23):
            party = row_leg[f"Nuance {i}"].iloc[0]
            if str(party) == "nan":
                continue
            alliance = ALLIANCES_PARTY[party]
            df_candidats.at[index, f"Votes {alliance}"] += round(row_leg[f"Voix {i}"].iloc[0] * pourcentage)

print(df_candidats["Votes Alliance gauche"])

0      11702
1      14561
2      12861
3      11157
4      10019
       ...  
567    17204
568     4970
569    11010
570     8116
571    10580
Name: Votes Alliance gauche, Length: 572, dtype: object


### 3.3 Isoler les deux premiers candidats

In [333]:
df_candidats_second = pd.DataFrame(columns=[
    "Libellé du département",
    "Libellé de la circonscription",
    "Parti 1",
    "Voix 1",
    "Parti 2",
    "Voix 2"
])

for index, row in df_candidats.iterrows():
    # Get the votes for each candidate
    votes = {alliance: row[f"Votes {alliance}"] for alliance in ALLIANCES_PARTY.values()}
    # Sort the candidates based on the number of votes
    sorted_votes = sorted(votes.items(), key=lambda item: item[1], reverse=True)
    # Get the top two candidates and their votes
    parti_1, voix_1 = sorted_votes[0]
    parti_2, voix_2 = sorted_votes[1]

    
    # Add a new row to the new DataFrame
    new_row = {
        "Libellé du département": row["Libellé du département"],
        "Libellé de la circonscription": row["Libellé de la circonscription"],
        "Parti 1": parti_1,
        "Voix 1": 0,
        "Parti 2": parti_2,
        "Voix 2": 0
    }
    df_candidats_second = pd.concat([df_candidats_second, pd.DataFrame([new_row])], ignore_index=True)

df_candidats_second['VS'] = df_candidats_second.apply(lambda row: ' vs '.join(sorted([row['Parti 1'], row['Parti 2']])), axis=1)
print(df_candidats_second['VS'].value_counts())

[('Alliance droite', 21198), ('Alliance gauche', 11702), ('Macron', 6555), ('Extrème droite', 3655), ('Extrème gauche', 1945), ('Divers', 1832)]
[('Alliance droite', 25467), ('Alliance gauche', 14561), ('Macron', 8441), ('Extrème droite', 4354), ('Extrème gauche', 2296), ('Divers', 1843)]
[('Alliance droite', 14567), ('Alliance gauche', 12861), ('Macron', 7342), ('Extrème droite', 3749), ('Extrème gauche', 1758), ('Divers', 1603)]
[('Alliance droite', 25166), ('Alliance gauche', 11157), ('Macron', 7449), ('Extrème droite', 4044), ('Extrème gauche', 1946), ('Divers', 1862)]
[('Alliance droite', 18446), ('Alliance gauche', 10019), ('Macron', 4830), ('Extrème droite', 3332), ('Extrème gauche', 1894), ('Divers', 1649)]
[('Alliance droite', 20633), ('Alliance gauche', 6058), ('Macron', 4277), ('Extrème droite', 2269), ('Extrème gauche', 2135), ('Divers', 1258)]
[('Alliance droite', 20108), ('Alliance gauche', 5897), ('Macron', 4179), ('Extrème droite', 2431), ('Extrème gauche', 2109), ('Div

### 3.4 Votes du second tour

Hypothèse : 

- La gauche s'unit définitivement
- La droite s'unit définitivement
- Si Macron - Gauche, alors la droite vote Macron
- Si Macron - Droite, alors la droite vote Macron
- Si Gauche - Droite, alors Les votes de Macron vont à 50% de chaque côté

In [343]:
MACRON_GAUCHE = 0.5
MACRON_DROITE = 0.5

TABLEAU_DES_REPORTS = {
    # Premier candidat :
    "Extrème gauche" : {
        # Deuxième candidat :          # Qui vote pour le premier
        "Alliance gauche" : {"Extrème gauche" : 1, "Alliance gauche" : 0, "Macron" : 0, "Alliance droite" : 0, "Extrème droite" : 0, "Divers" : 0.5},
        "Macron" : {"Extrème gauche" : 1, "Alliance gauche" : 1, "Macron" : 0, "Alliance droite" : 0, "Extrème droite" : 0, "Divers" : 0.5},
        "Alliance droite" : {"Extrème gauche" : 1, "Alliance gauche" : 1, "Macron" : 0, "Alliance droite" : 0, "Extrème droite" : 0, "Divers" : 0.5},
        "Extrème droite" : {"Extrème gauche" : 1, "Alliance gauche" : 1, "Macron" : MACRON_GAUCHE, "Alliance droite" : 0, "Extrème droite" : 0, "Divers" : 0.5},
        "Divers" : {"Extrème gauche" : 1, "Alliance gauche" : 1, "Macron" : 0.5, "Alliance droite" : 0.5, "Extrème droite" : 0.5, "Divers" : 0},
    },
    "Alliance gauche" : {
        "Extrème gauche" : {"Extrème gauche" : 0, "Alliance gauche" : 1, "Macron" : 1, "Alliance droite" : 1, "Extrème droite" : 1, "Divers" : 0.5},
        "Macron" : {"Extrème gauche" : 1, "Alliance gauche" : 1, "Macron" : 0, "Alliance droite" : 0, "Extrème droite" : 0, "Divers" : 0.5},
        "Alliance droite" : {"Extrème gauche" : 1, "Alliance gauche" : 1, "Macron" : MACRON_GAUCHE, "Alliance droite" : 0, "Extrème droite" : 0, "Divers" : 0.5},
        "Extrème droite" : {"Extrème gauche" : 1, "Alliance gauche" : 1, "Macron" : 1, "Alliance droite" : 0, "Extrème droite" : 0, "Divers" : 0},
        "Divers" : {"Extrème gauche" : 1, "Alliance gauche" : 1, "Macron" : 0.5, "Alliance droite" : 0.5, "Extrème droite" : 0.5, "Divers" : 0},
    },
    "Macron" : {
        "Extrème gauche" : {"Extrème gauche" : 0, "Alliance gauche" : 0, "Macron" : 1, "Alliance droite" : 1, "Extrème droite" : 1, "Divers" : 0.5},
        "Alliance gauche" : {"Extrème gauche" : 0, "Alliance gauche" : 0, "Macron" : 1, "Alliance droite" : 1, "Extrème droite" : 1, "Divers" : 0.5},
        "Alliance droite" : {"Extrème gauche" : 1, "Alliance gauche" : 1, "Macron" : 1, "Alliance droite" : 0, "Extrème droite" : 0, "Divers" : 0.5},
        "Extrème droite" : {"Extrème gauche" : 1, "Alliance gauche" : 1, "Macron" : 1, "Alliance droite" : 0, "Extrème droite" : 0, "Divers" : 0.5},
        "Divers" : {"Extrème gauche" : 0.5, "Alliance gauche" : 0.5, "Macron" : 0, "Alliance droite" : 0.5, "Extrème droite" : 0.5, "Divers" : 0},
    },
    "Alliance droite" : {
        "Extrème gauche" : {"Extrème gauche" : 0, "Alliance gauche" : 0, "Macron" : 1, "Alliance droite" : 1, "Extrème droite" : 1, "Divers" : 0.5},
        "Alliance gauche" : {"Extrème gauche" : 0, "Alliance gauche" : 0, "Macron" : MACRON_DROITE, "Alliance droite" : 1, "Extrème droite" : 1, "Divers" : 0.5},
        "Macron" : {"Extrème gauche" : 0, "Alliance gauche" : 0, "Macron" : 0, "Alliance droite" : 1, "Extrème droite" : 1, "Divers" : 0.5},
        "Extrème droite" : {"Extrème gauche" : 1, "Alliance gauche" : 1, "Macron" : 1, "Alliance droite" : 1, "Extrème droite" : 0, "Divers" : 0.5},
        "Divers" : {"Extrème gauche" : 0.5, "Alliance gauche" : 0.5, "Macron" : 0.5, "Alliance droite" : 1, "Extrème droite" : 1, "Divers" : 0},
    },
    "Extrème droite" : {
        "Extrème gauche" : {"Extrème gauche" : 0, "Alliance gauche" : 0, "Macron" : MACRON_DROITE, "Alliance droite" : 1, "Extrème droite" : 1, "Divers" : 0.5},
        "Alliance gauche" : {"Extrème gauche" : 0, "Alliance gauche" : 0, "Macron" : 0, "Alliance droite" : 1, "Extrème droite" : 1, "Divers" : 0.5},
        "Macron" : {"Extrème gauche" : 0, "Alliance gauche" : 0, "Macron" : 0, "Alliance droite" : 1, "Extrème droite" : 1, "Divers" : 0.5},
        "Alliance droite" : {"Extrème gauche" : 0, "Alliance gauche" : 0, "Macron" : 0, "Alliance droite" : 0, "Extrème droite" : 1, "Divers" : 0.5},
        "Divers" : {"Extrème gauche" : 0.5, "Alliance gauche" : 0.5, "Macron" : 0.5, "Alliance droite" : 1, "Extrème droite" : 1, "Divers" : 0},
    },
    "Divers" : {
        "Extrème gauche" : {"Extrème gauche" : 0, "Alliance gauche" : 0, "Macron" : 0.5, "Alliance droite" : 0.5, "Extrème droite" : 0.5, "Divers" : 1},
        "Alliance gauche" : {"Extrème gauche" : 0, "Alliance gauche" : 0, "Macron" : 0.5, "Alliance droite" : 0.5, "Extrème droite" : 0.5, "Divers" : 1},
        "Macron" : {"Extrème gauche" : 0.5, "Alliance gauche" : 0.5, "Macron" : 0, "Alliance droite" : 0.5, "Extrème droite" : 0.5, "Divers" : 1},
        "Alliance droite" : {"Extrème gauche" : 0.5, "Alliance gauche" : 0.5, "Macron" : 0.5, "Alliance droite" : 0, "Extrème droite" : 0, "Divers" : 1},
        "Extrème droite" : {"Extrème gauche" : 0.5, "Alliance gauche" : 0.5, "Macron" : 0.5, "Alliance droite" : 0, "Extrème droite" : 0, "Divers" : 1},
    }
}

In [344]:
import numpy as np
# Reset the votes : 
df_candidats_second["Voix 1"] = 0
df_candidats_second["Voix 2"] = 0

for index, row in df_candidats_second.iterrows():
    # Get the 2 parties
    parti_1 = row["Parti 1"]
    parti_2 = row["Parti 2"]

    # Get the initial votes of the candidates
    row_candidate = df_candidats.loc[(df_candidats["Libellé du département"] == row["Libellé du département"]) & (df_candidats["Libellé de la circonscription"] == row["Libellé de la circonscription"])]
    unique_values = set(ALLIANCES_PARTY.values())
    for alliance in unique_values:
        votes = row_candidate[f"Votes {alliance}"].iloc[0]
        # Get the report for each candidate
        report_pour_1 = TABLEAU_DES_REPORTS[parti_1][parti_2][alliance]
        report_pour_2 = TABLEAU_DES_REPORTS[parti_2][parti_1][alliance]

        # Add the votes to the DataFrame
        df_candidats_second.at[index, f"Voix 1"] += round(votes * report_pour_1)
        df_candidats_second.at[index, f"Voix 2"] += round(votes * report_pour_2)

# Determine which candidate has the highest score
df_candidats_second['Winner'] = np.where(df_candidats_second['Voix 1'] > df_candidats_second['Voix 2'], df_candidats_second['Parti 1'], df_candidats_second['Parti 2'])

print(df_candidats_second["Winner"].value_counts())

Winner
Alliance gauche    287
Alliance droite    269
Macron              11
Extrème gauche       5
Name: count, dtype: int64
